In [5]:
%load_ext autoreload
%autoreload 2

# data processing
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
from sklearn.preprocessing import StandardScaler


# plotting
import seaborn as sns
import matplotlib.pyplot as plt

# Statistics (correlation)
from scipy.stats import pearsonr, spearmanr

# libraries for the model
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, precision_score, recall_score

In [6]:
# custom functions
from pt_helpers import *

## Load the training data
This is the clean data that we processed in the notebook `Training_data_preparation`

In [11]:
data_path = "DATA/training_data/"

df = pd.read_csv(data_path + "VEX_edac_mag_labeled.csv")

In [12]:
df.head()

,DATE_TIME,EDAC,BX,BY,BZ,BT,XSC,YSC,ZSC,RSC,cme
0,2006-04-24 00:00:00,0,12.357658,-1.304164,-3.332425,12.881274,-4451.055178,-3196.485753,-65466.762260,65695.760575,0
1,2006-04-24 00:05:00,0,12.868947,-0.980800,-3.360027,13.340680,-4202.246280,-3138.377907,-65806.350827,66015.078600,0
2,2006-04-24 00:10:00,0,12.857438,-0.871986,-3.487877,13.355384,-3954.000329,-3080.233288,-66137.913808,66327.612616,0
3,2006-04-24 00:15:00,0,12.898635,-0.684986,-2.885689,13.248405,-3705.057257,-3021.761270,-66463.291041,66635.079608,0
4,2006-04-24 00:20:00,0,12.766473,-0.517608,-2.217135,12.972905,-3453.676541,-2962.553108,-66784.717784,66939.596338,0


We separate the feature matrix `X` and the labels `y`.
- `X` will contain X Y and Z coordinates of the magnetic field

In [14]:
y = df['cme'].values
# Normalize the features
X = df[['BT', 'RSC']].values

In [15]:
print("Shape of y:", y.shape)
print("Shape of X:", X.shape)

Shape of y: (904224,)
Shape of X: (904224, 2)


## LSTM

In [24]:
time_steps = 36  # 3 hours of data
X_seq, y_seq = create_sequences(X, y, time_steps)
print("Shape of X and y sequences:", X_seq.shape, y_seq.shape)
# Drop all row containing nans
X_seq = X_seq[~np.isnan(X_seq).any(axis=-1)]
X_seq = X_seq.reshape(-1, time_steps, X_seq.shape[1])
y_seq = y_seq[~np.isnan(y_seq)]
print("Shape of X and y sequences after dropping nans:", X_seq.shape, y_seq.shape)
print("Sum y_seq:", np.sum(y_seq))

Sum y: nan
Shape of X and y sequences: (904188, 36, 2) (904188,)
Shape of X and y sequences after dropping nans: (893033, 36, 2) (893033,)
Sum y_seq: 15939.0


In [80]:
class CMEPredictor(nn.Module):
    def __init__(self, n_features, n_hidden=50, n_layers=2):
        super(CMEPredictor, self).__init__()
        self.lstm = nn.LSTM(input_size=n_features, hidden_size=n_hidden, num_layers=n_layers, batch_first=True)
        self.linear = nn.Linear(n_hidden, 1)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x[:, -1, :])
        return torch.sigmoid(x)

n_features = X_seq.shape[2]  # Number of features in the dataset
model = CMEPredictor(n_features)

In [81]:
X_train, X_val, y_train, y_val = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

# # Standardize the data
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train.reshape(-1, n_features)).reshape(-1, time_steps, n_features)
# X_val = scaler.transform(X_val.reshape(-1, n_features)).reshape(-1, time_steps, n_features)

# Convert to PyTorch tensors
train_data = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
val_data = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32))

In [82]:
# Create DataLoaders
batch_size = 32  # Adjust this based on your system's capability
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size)

In [83]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
n_epochs = 10

### Training and evaluation

In [84]:
# Training loop with reduced batch size and data loading on-the-fly
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch.view(-1, 1))
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for X_batch, y_batch in val_loader:
            val_output = model(X_batch)
            val_loss += criterion(val_output, y_batch.view(-1, 1)).item()
    val_loss /= len(val_loader)
    
    print(f'Epoch {epoch+1}, Loss: {loss.item()}, Val Loss: {val_loss}')

Epoch 1, Loss: 0.017840756103396416, Val Loss: 0.06819538405116359



KeyboardInterrupt



In [ ]:
model.eval()
predictions = []
true_labels = []
with torch.no_grad():
    for X_batch, y_batch in val_loader:
        outputs = model(X_batch)
        predicted = (outputs > 0.5).float()  # Using 0.5 as the threshold for binary classification
        predictions.extend(predicted.view(-1).tolist())
        true_labels.extend(y_batch.tolist())

# Calculate metrics
accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)
precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')

## LSTM with class weights

In [ ]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = torch.tensor(class_weights, dtype=torch.float32)

# Use the class weights in your loss function
criterion = nn.BCELoss(weight=class_weights[1])  

In [ ]:
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch.view(-1, 1))
        loss.backward()
        optimizer.step()

        # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for X_batch, y_batch in val_loader:
            val_output = model(X_batch)
            val_loss += criterion(val_output, y_batch.view(-1, 1)).item()
    val_loss /= len(val_loader)
    
    print(f'Epoch {epoch+1}, Loss: {loss.item()}, Val Loss: {val_loss}')

In [ ]:
model.eval()
predictions = []
true_labels = []
with torch.no_grad():
    for X_batch, y_batch in val_loader:
        outputs = model(X_batch)
        predicted = (outputs > 0.5).float()  # Using 0.5 as the threshold for binary classification
        predictions.extend(predicted.view(-1).tolist())
        true_labels.extend(y_batch.tolist())

# Calculate metrics
accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)
precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')